In [ ]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


In [2]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("google/vit-large-patch16-224-in21k", use_fast=True)
model = AutoModelForImageClassification.from_pretrained("google/vit-large-patch16-224-in21k", use_auth_token=os.environ['HF_TOKEN'])

/root/miniconda3/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-large-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import torch
import torch.nn as nn

model.classifier = torch.nn.Linear(model.config.hidden_size, 1000)  # ImageNet 有 1000 个类别
model.config.num_labels = 1000

In [4]:
print(model.classifier.weight.shape)
print(model.classifier.bias.shape)


torch.Size([1000, 1024])
torch.Size([1000])


In [5]:
# print the model 
print(model)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-23): 24 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=1024, out_features=4096, bias=True)
           

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-23): 24 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=1024, out_features=4096, bias=True)
           

In [7]:
from peft import LoraConfig, get_peft_model, TaskType   

target_modules = ['query', 'value', 'key', 'dense', 'intermediate.dense', 'output.dense']


lora_config = LoraConfig(
    # 分类任务类型
    task_type=TaskType.FEATURE_EXTRACTION, # 或 TaskType.IMAGE_CLASSIFICATION (取决于 PEFT 版本)
    r=256,          # LoRA 秩 (Rank): 决定引入的参数量和表达能力
    lora_alpha=256, # 缩放因子 (通常设为 r 的两倍或相等)
    target_modules=target_modules,
    lora_dropout=0.05,
    bias="none"
)


model_peft = get_peft_model(model, lora_config)

print(f"--- LoRA Model Summary ---")
model_peft.print_trainable_parameters()

--- LoRA Model Summary ---
trainable params: 113,246,208 || all params: 417,572,840 || trainable%: 27.1201


In [8]:
for name, param in model_peft.named_parameters():
    if "lora_" not in name and "classifier" not in name:
        param.requires_grad = False


In [9]:
print(model_peft)

PeftModelForFeatureExtraction(
  (base_model): LoraModel(
    (model): ViTForImageClassification(
      (vit): ViTModel(
        (embeddings): ViTEmbeddings(
          (patch_embeddings): ViTPatchEmbeddings(
            (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): ViTEncoder(
          (layer): ModuleList(
            (0-23): 24 x ViTLayer(
              (attention): ViTAttention(
                (attention): ViTSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=256, bias=False)
                    )
                    (lora_B): ModuleDi

In [10]:
model_peft.base_model.vit.encoder.gradient_checkpointing = True  # 启用梯度检查点以节省内存
model_peft.base_model.vit.use_flash_attention = True      # 启用 Flash Attention 提升速度（如果支持）

try:
    model.enable_xformers_memory_efficient_attention()
except:
    print("xformers unavailable — fallback to FlashAttention only.")

xformers unavailable — fallback to FlashAttention only.


## Benchmark the base model performance on ImageNet-Mini

In [11]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

# --- 1. 路径配置 ---
# 假设您的训练集和验证集路径如下
TRAIN_DIR = "/root/autodl-tmp/imagenet/train"
VAL_DIR = "/root/autodl-tmp/imagenet/val"

# --- 2. 预处理 (Transforms) ---
# ImageNet 标准归一化参数
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# 模型输入尺寸与归一化保持一致
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandAugment(num_ops=2, magnitude=9),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    ),
    transforms.RandomErasing(p=0.25)
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225)
    ),
])

# --- 3. 辅助函数 (解决 .JPEG 大小写问题) ---
def is_valid_image(path):
    """
    自定义函数，确保只加载有效图片文件 (包括大写 .JPEG)
    """
    return path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))

# --- 4. 加载 Datasets (使用 ImageFolder) ---

print("正在加载训练集...")
try:
    train_dataset = ImageFolder(
        root=TRAIN_DIR,
        transform=train_transform,
        is_valid_file=is_valid_image # 使用自定义函数处理大小写
    )
    
    print(f"训练集加载成功: {len(train_dataset)} 张图片, {len(train_dataset.classes)} 个类别。")

    print("\n正在加载验证集...")
    val_dataset = ImageFolder(
        root=VAL_DIR,
        transform=val_transform,
        is_valid_file=is_valid_image # 同样使用
    )
    
    print(f"验证集加载成功: {len(val_dataset)} 张图片, {len(val_dataset.classes)} 个类别。")

    # --- 5. 创建 DataLoaders ---
    BATCH_SIZE = 64 # 根据您的 GPU 显存调整 (例如 64, 128, 256)
    NUM_WORKERS = 8  # 根据您的 CPU 核心数调整 (例如 4, 8, 16)

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,      # 训练集需要打乱
        num_workers=NUM_WORKERS,
        pin_memory=True,   # 锁定内存以加快 GPU 传输
        drop_last=True     # 丢弃最后一个不完整的批次
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=32,
        shuffle=False,     # 验证集不需要打乱
        num_workers=NUM_WORKERS,
        pin_memory=True
    )

    print(f"\n✅ DataLoader 准备就绪 (Batch Size: {BATCH_SIZE})")

    # (可选) 检查类别映射是否一致
    # print(train_dataset.class_to_idx)
    
except FileNotFoundError as e:
    print(f"❌ 错误: 找不到数据目录。请检查路径配置: {e}")
except Exception as e:
    print(f"❌ 加载数据时发生错误: {e}")

正在加载训练集...
训练集加载成功: 1281167 张图片, 1000 个类别。

正在加载验证集...
验证集加载成功: 50000 张图片, 1000 个类别。

✅ DataLoader 准备就绪 (Batch Size: 64)


In [12]:
# --------------------------------------
# 4️⃣ 验证函数（Top-1 + Top-5）
# --------------------------------------
def evaluate_model(model, data_loader, device):
    model.eval()
    correct1 = 0
    correct5 = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating", unit="batch"):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(pixel_values=images)
            logits = outputs.logits

            # Top-1
            _, pred = logits.max(dim=1)
            correct1 += (pred == labels).sum().item()

            # Top-5
            top5 = logits.topk(5, dim=1).indices
            correct5 += (top5 == labels.unsqueeze(1)).any(dim=1).sum().item()

            total += labels.size(0)

    top1 = 100 * correct1 / total
    top5 = 100 * correct5 / total
    return top1, top5

In [13]:
# acc = evaluate_model(model_peft, val_loader, device)
# print(f"\n--- LoRA Model on ImageNet Validation Set ---")
# print(f"Accuracy: {acc:.2f}%")

```shell
--- LoRA Model on ImageNet Validation Set ---
Accuracy: 0.072%
```

## Train model to improve performance further with ImageNet dataset

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from torch.optim import AdamW
import copy

# ====================================================
# 1️⃣ 训练配置
# ====================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter(log_dir="/root/tf-logs")

num_epochs = 10
warmup_epochs = 1
total_steps = len(train_loader) * num_epochs
warmup_steps = len(train_loader) * warmup_epochs
final_lr = 1e-6   # 👈 不降到 0，更稳定
base_lr = 1e-4
ema_decay = 0.9997

# compile (保持全局可优化且不损坏 PEFT)
model_peft = torch.compile(model_peft, mode="max-autotune", fullgraph=False)
criterion = nn.CrossEntropyLoss().to(device)

optimizer = AdamW(
    filter(lambda p: p.requires_grad, model_peft.parameters()),
    lr=base_lr,
    betas=(0.85, 0.98),
    weight_decay=0.01
)

# ====================================================
# 2️⃣ EMA 模块
# ====================================================
# 创建独立 EMA 权重副本
ema_model = copy.deepcopy(model_peft).eval()
for p in ema_model.parameters():
    p.requires_grad_(False)

def ema_update(model, ema_model, decay):
    with torch.no_grad():
        model_params = dict(model.named_parameters())
        ema_params = dict(ema_model.named_parameters())

        for k in model_params.keys():
            if model_params[k].dtype.is_floating_point:
                ema_params[k].data.mul_(decay).add_(model_params[k].data, alpha=1-decay)

# ====================================================
# 3️⃣ Warmup + Linear Decay 学习率函数
# ====================================================
def get_lr(step):
    if step < warmup_steps:
        return base_lr * (step / warmup_steps)  # linear warmup
    else:
        decay_ratio = (step - warmup_steps) / (total_steps - warmup_steps)
        return final_lr + (base_lr - final_lr) * (1 - decay_ratio)

# ====================================================
# 4️⃣ 训练循环
# ====================================================
use_bf16 = torch.cuda.is_bf16_supported()
scaler = torch.amp.GradScaler(enabled=not use_bf16)

best_top1 = 0
patience = 3
bad_epochs = 0
global_step = 0

for epoch in range(num_epochs):
    model_peft.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")

    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad(set_to_none=True)

        # 更新学习率
        lr = get_lr(global_step)
        for pg in optimizer.param_groups:
            pg["lr"] = lr

        with torch.amp.autocast(device_type="cuda", dtype=torch.bfloat16 if use_bf16 else torch.float16):
            outputs = model_peft(pixel_values=images)
            loss = criterion(outputs.logits, labels)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model_peft.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        # EMA 更新
        ema_update(model_peft, ema_model, ema_decay)

        running_loss += loss.item()
        writer.add_scalar("Train/Loss", loss.item(), global_step)
        writer.add_scalar("LR", lr, global_step)

        global_step += 1
        pbar.set_postfix(loss=f"{loss.item():.4f}", lr=f"{lr:.2e}")

    # =============== 验证：使用 EMA 模型更稳 ===============
    top1, top5 = evaluate_model(ema_model, val_loader, device)
    print(f"Validation Top1={top1:.2f}% | Top5={top5:.2f}%")

    if top1 > best_top1:
        best_top1 = top1
        bad_epochs = 0
        print(f"🟢 Best updated → {best_top1:.2f}%, saving weights")

        # 保存 LoRA
        model_peft.save_pretrained("vit_lora_r256_ema_best")

        # 保存分类头
        torch.save(
            model_peft.base_model.classifier.state_dict(),
            "vit_classifier_r256_ema_best.pt"
        )

        # 保存 EMA 完整推理模型（可选）
        torch.save(ema_model.state_dict(), "vit_full_ema_state.pt")

    else:
        bad_epochs += 1
        print(f"⚠ {bad_epochs} bad epoch(s)")

    if bad_epochs >= patience:
        print("⛔ Early stopping")
        break

writer.close()


NameError: name 'copy' is not defined